In [17]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import figure
from torch.utils.data import DataLoader
import plain_inn_swap
from tesladata import TeslaDataset
import numpy as np

### Constants

In [2]:
batch_size = 2048
lr = 1e-5
device = "cuda:0"

num_features = 100
num_blocks = 5

# Lagrange multipliers of our multi-objective function
#Todo: those have to be optimized via wandb
lambd_predict_back= 1. 
lambd_latent = 1. #300.
lambd_rev = 1. #450.

### Training
The following cell basically trains an Invertible Neural Network as of (Ardizzone, 2018) without padding: `[y,z] <=> x` with `x` = parameters, `y` = temperature readings and `z` our latent variable (iid. Gaussian)

In [3]:
ds = TeslaDataset()
train_dataloader = DataLoader(ds, batch_size=batch_size, shuffle=False)
# get one sample of our dataset to infer its input and output dimension
x,y = ds.__getitem__(0)
dim_inp = x.shape[0]
dim_outp = 1

In [4]:
my_inn = plain_inn_swap.INN(ndim_tot = dim_inp, ndim_y = dim_outp, ndim_x = dim_inp, ndim_z=dim_inp-dim_outp, 
            device = device,
             lambd_predict_back = lambd_predict_back, lambd_latent = lambd_latent, lambd_rev = lambd_rev,
             feature = num_features, num_blocks = num_blocks, batch_size = batch_size, lr = lr)

In [5]:
my_inn.train(n_epochs=100, train_loader=train_dataloader, val_loader=train_dataloader)

0 Loss: 1041.0696507147213 l_x_2: 618.7860115843082 l_y: 421.86192123058737 l_z: 0.21180451613500578 l_x_1: 0.2099133836908625
val loss x: 2510.520263671875 val loss y 426.5432434082031
1 Loss: 840.8490382564861 l_x_2: 419.13453338780533 l_y: 421.2789866333708 l_z: 0.21106478783789032 l_x_1: 0.22445344747206486
2 Loss: 727.6872338809825 l_x_2: 293.3351350661811 l_y: 433.9116251359292 l_z: 0.21030359448642905 l_x_1: 0.2301700843856969
3 Loss: 695.5470655527 l_x_2: 233.14263715656526 l_y: 461.9646711087008 l_z: 0.21019357319818724 l_x_1: 0.22956371423574762
4 Loss: 697.5919768506781 l_x_2: 209.77906400785534 l_y: 487.3738358523868 l_z: 0.21012474801562248 l_x_1: 0.22895224242035403
5 Loss: 698.4956211923801 l_x_2: 198.06908038778042 l_y: 499.9880473985585 l_z: 0.21043018713046652 l_x_1: 0.22806321891075973
val loss x: 2500.907958984375 val loss y 534.0298461914062
6 Loss: 696.0133443435808 l_x_2: 190.80692977205328 l_y: 504.76780438204423 l_z: 0.21061255697400197 l_x_1: 0.227997632509251

KeyboardInterrupt: 

### Analysis of the model

In [21]:
x,y = next(iter(train_dataloader))

# Make a prediction
pred,_ = my_inn.model(x)
pred = pred.detach().cpu().numpy()[:,0] # take temperature components, the other three elements are z

y = y.cpu().numpy()
#print(np.concatenate([df_y_tensor_np, pred],axis = 1))
print('relative l2 norm: ', np.linalg.norm(y-pred) / np.linalg.norm(y))

relative l2 norm:  993.0346


In [23]:
figure(figsize=(10, 8), dpi= 1440)

plt.plot(pred, '--')
plt.plot(y, '-')
plt.legend(['Prediction', 'ground-truth'])
plt.ylabel('Temperature / °C')
plt.show()